In [3]:
import pandas as pd
df = pd.read_csv("ecommerceDataset.csv",names=['category','description'],header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


In [4]:
df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

In [5]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [7]:
df.category.replace("Clothing & Accessories","Clothing_Accessories",inplace=True)
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [8]:
#Adding '__label__' as a prefix
df['category'] = "__label__" + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [23]:
#Merging category and description column
df['category_description'] = df['category'] + " " + df['description']
df.head(5)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [24]:
df['category_description'][0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

In [25]:
import re
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()
#viki's bookcase bookshelf 3 shelf shelve white hi

In [26]:
preprocess("  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi")

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [27]:
df["category_description"]= df["category_description"].map(preprocess)

In [20]:
df.head(5)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__householdpaper plane design framed wa...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__householdsaf 'floral' framed painting...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__householdsaf 'uv textured modern art ...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__householdsaf flower print framed pain...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__householdincredible gifts india woode...


In [28]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=0.2)

In [29]:
train.shape,test.shape

((40339, 3), (10085, 3))

In [31]:
test.head(5)

,category,description,category_description
37167,__label__Clothing_Accessories,Clovia Women's Plain Thigh Slimmer This amazin...,__label__clothing_accessories clovia women's p...
45763,__label__Electronics,Gizga Essentials Professional Lens Pen Cleanin...,__label__electronics gizga essentials professi...
40850,__label__Electronics,ZOTAC GeForce GTX 1050 Ti mini ZT-P10510A-10L ...,__label__electronics zotac geforce gtx 1050 ti...
20161,__label__Books,Indra Nooyi - A Biography About the Author Abo...,__label__books indra nooyi a biography about t...
19604,__label__Books,Standard Wire Bound Manuscript Paper: Green Co...,__label__books standard wire bound manuscript ...


In [34]:
train.to_csv("ecommerce.train",columns=["category_description"],index=False,header=False)
train.to_csv("ecommerce.test",columns=["category_description"],index=False,header=False)

In [37]:
#fasttext.train_supervised expects the file in a certain format
#Can do text classification at the sa
import fasttext
model = fasttext.train_supervised(input = 'ecommerce.train')
model.test("ecommerce.test")

Read 4M words
Number of words:  79503
Number of labels: 4
Progress: 100.0% words/sec/thread: 2704486 lr:  0.000000 avg.loss:  0.178954 ETA:   0h 0m 0s2704614 lr: -0.000008 avg.loss:  0.178954 ETA:   0h 0m 0s


(40335, 0.9824470063220528, 0.9824470063220528)

In [38]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99825871]))

In [39]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000989]))

In [41]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super comb")

(('__label__clothing_accessories',), array([1.00001001]))